# Selecting which parameters to fit

For complex simulations, there may be a lot of parameters; [a simple argon simulation](../../../tutorials/Argon-a-to-z.ipynb) has two parameters, and later on in this guide we will see a water simulation with a total of 8 - this only gets bigger as the molecules and forces become more complex.

Thus, a user may only want to refine a subset of all their parameters. In MDMC this can be done in two ways:

### Fixing a Parameter

`Parameter` objects can be fixed.  This can either be set when they are initialised (created) or changed for an existing `Parameter`.  By default, an initialised `Parameter` is **not** fixed.

In [1]:
from MDMC.MD.parameters import Parameter
charge = Parameter(value=0.5, name='charge', fixed=True, unit='e')

Attempting to change a fixed parameter produces a warning and does **not** change the parameter.

In [2]:
charge.value = 12
print(f"The value of the charge parameter is: {charge.value}")

The value of the charge parameter is: 0.5 e


For parameters which already exist (for example, a parameter created by a potential), the parameter can be fixed by setting their `fixed` attribute to `True`.

In [3]:
sigma = Parameter(1.0, name='sigma', unit='Ang')
print('Is sigma fixed: {}'.format(sigma.fixed))
sigma.fixed = True
print('Is sigma fixed: {}'.format(sigma.fixed))

Is sigma fixed: False
Is sigma fixed: True


### Filtering parameters

Only parameters that are passed to `Control` (as `fit_parameters`) will be refined.  While it is simplest to pass all parameters in a `Universe` to `Control`, it is also possible to filter out a subset.  To demonstrate this, we use a universe filled with water under a SPCE force field, as used in [running a simulation](running-a-simulation.ipynb).

In [4]:
from MDMC.MD import *

universe = Universe(dimensions=21.75, constraint_algorithm=Shake(1e-4, 100), electrostatic_solver=PPPM(accuracy=1e-5))
H1 = Atom('H')
H2 = Atom('H', position=(0., 1.63298, 0.))
O = Atom('O', position=(0., 0.81649, 0.57736))
H_coulombic = Coulombic(atoms=[H1, H2], cutoff=10.)
O_coulombic = Coulombic(atoms=O, cutoff=10.)
water_mol = Molecule(position=(0, 0, 0),
                     velocity=(0, 0, 0),
                     atoms=[H1, H2, O],
                     interactions=[Bond((H1, O), (H2, O), constrained=True),
                                   BondAngle(H1, O, H2, constrained=True)],
                     name='water')
universe.fill(water_mol, num_density=0.03356718472021752)
O_dispersion = Dispersion(universe, [O.atom_type, O.atom_type], cutoff=10., vdw_tail_correction=True)
universe.add_force_field('SPCE')

Universe created with:
Dimensions [21.75 21.75 21.75]


There are 8 parameters in the universe:

In [5]:
print(universe.parameters)

{'equilibrium_state (#23)': <Parameter
 {ID: 23,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#24)': <Parameter
 {ID: 24,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'equilibrium_state (#9)': <Parameter
 {ID: 9,
  type: 'equilibrium_state',
  value: 109.47 deg,
  unit: 'deg',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#10)': <Parameter
 {ID: 10,
  type: 'potential_strength',
  value: 383.0 kJ / mol rad ^ 2,
  unit: 'kJ / mol rad ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential',
  tied: False}>,

So while all 8 parameters can be passed when initiliasing `Control`, they can also be filtered. `Parameters` objects have a number of convenience methods to assist with this:

In [6]:
parameters = universe.parameters
help(parameters)

Help on Parameters in module MDMC.MD.parameters object:

class Parameters(builtins.dict)
 |  Parameters(init_parameters: "Optional[Union[Parameter, 'list[Parameter]']]" = None)
 |  
 |  A `dict-like` object where every element is a ``Parameter`` indexed by name,
 |  which contains a number of helper methods for filtering.
 |  
 |  Although ``Parameters`` is a `dict`, it should be treated like a `list` when writing to it;
 |  i.e. initialise it using a `list` and use `append` to add to it. These parameters can
 |  then be accessed by their name as a key.
 |  
 |  In short; Parameters writes like a list and reads like a dict.
 |  
 |  Parameters
 |  ----------
 |  init_parameters: ``Parameter`` or `list` of ``Parameter``s, optional, default None
 |      The initial ``Parameter`` objects that the ``Parameters`` object contains.
 |  
 |  Attributes
 |  ----------
 |  array: np.ndarray
 |      An alphabetically-sorted numpy array of the ``Parameter``s stored in this object.
 |  
 |  Method 

For example, if only the charge parameters should be refined, `parameters` could be filtered by name:

In [7]:
charges = parameters.filter_name('charge')
print(charges)

# charges is a Parameters object
print('\nThe class of charges is: {}'.format(type(charges)))

{'charge (#28)': <Parameter
 {ID: 28,
  type: 'charge',
  value: 0.4238 e,
  unit: 'e',
  fixed: False,
  constraints: None,
  interactions_name: 'Coulombic',
  functions_name: 'Coulomb',
  tied: False}>, 'charge (#11)': <Parameter
 {ID: 11,
  type: 'charge',
  value: -0.8476 e,
  unit: 'e',
  fixed: False,
  constraints: None,
  interactions_name: 'Coulombic',
  functions_name: 'Coulomb',
  tied: False}>}

The class of charges is: <class 'MDMC.MD.parameters.Parameters'>


As each filter returns a `Parameters` object, filters can be chained together. For example, to find the potential strengths of all bonds:

In [8]:
bond_potential_strengths = parameters.filter_name('potential_strength').filter_interaction('Bond')
print(bond_potential_strengths)

# These operations are commutative
print('\nThe order these methods are'
      ' applied does not matter: {}'.format(bond_potential_strengths
                                             == parameters.filter_interaction('Bond').filter_name('potential_strength')))

{'potential_strength (#24)': <Parameter
 {ID: 24,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>}

The order these methods are applied does not matter: True


It is also possible to filter parameters based on the properties of the atoms to which they apply.  For instance, we can filter the SPCE parameters so that only parameters of interactions on H atoms are shown:

In [9]:
H_parameters = parameters.filter_atom_attribute('name', 'H')
print(H_parameters)

{'equilibrium_state (#23)': <Parameter
 {ID: 23,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#24)': <Parameter
 {ID: 24,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'equilibrium_state (#9)': <Parameter
 {ID: 9,
  type: 'equilibrium_state',
  value: 109.47 deg,
  unit: 'deg',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#10)': <Parameter
 {ID: 10,
  type: 'potential_strength',
  value: 383.0 kJ / mol rad ^ 2,
  unit: 'kJ / mol rad ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential',
  tied: False}>,

Finally there is also a more flexible method (`Parameters.filter`) which can be used in conjunction with any function to filter the parameters by some predicate function (a function which takes a parameter and returns True or False according to what is 'filtered out'). This is most easily done with a [lambda function](https://docs.python.org/3/howto/functional.html#small-functions-and-the-lambda-expression) but a regular function can also be used.

In [10]:
def is_length(parameter):
    """A function which tells you whether the parameter measures length or not."""
    return parameter.unit == 'Ang'

length_parameters = parameters.filter(is_length)
print(length_parameters)

# For those more familiar with Python, this can also be done using a lambda
lambda_length_parameters = parameters.filter(lambda p: p.unit == 'Ang')
print('\nThe same filter can be achieved using lambdas: {}'.format(lambda_length_parameters == length_parameters))

{'equilibrium_state (#23)': <Parameter
 {ID: 23,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'sigma (#38)': <Parameter
 {ID: 38,
  type: 'sigma',
  value: 3.166 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Dispersion',
  functions_name: 'LennardJones',
  tied: False}>}

The same filter can be achieved using lambdas: True
